In [1]:
import geopandas as gpd
from dask.distributed import Client, LocalCluster
from dask import delayed, dataframe as dd
from tqdm import tqdm
import dask

/home/virgilxw/miniconda3/envs/morph_model_v1_tess/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 13.0.0. Please consider upgrading.
  warnings.warn(


In [2]:
place = "glasgow"
local_crs = 27700

In [3]:
Daskcluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(Daskcluster)
client

2023-11-21 16:26:18,089 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-11-21 16:26:18,092 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-11-21 16:26:18,095 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-11-21 16:26:18,097 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-11-21 16:26:18,099 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-11-21 16:26:18,101 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-11-21 16:26:18,103 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-11-21 16:26:18,

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 250.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35417,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 250.47 GiB
Comm: tcp://127.0.0.1:36523,Total threads: 2
Dashboard: http://127.0.0.1:44439/status,Memory: 31.31 GiB
Nanny: tcp://127.0.0.1:39275,


In [4]:
tessellation = gpd.read_parquet(f"./output/{place}/tessellation_p1.pq").to_crs(local_crs)
buildings = gpd.read_parquet(f"./output/{place}/buildings_p1.pq").to_crs(local_crs)
streets = gpd.read_parquet(f"./output/{place}/streets_raw.pq").explode().to_crs(local_crs).reset_index(drop=True)

/tmp/ipykernel_2323/3257141034.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  streets = gpd.read_parquet(f"./output/{place}/streets_raw.pq").explode().to_crs(local_crs).reset_index(drop=True)


In [5]:
# Assuming tessellation is a GeoDataFrame
buffered_tessellation = tessellation.buffer(0.5)

In [6]:
tessellation_a = tessellation

tessellation_a["geometry"] = buffered_tessellation

buffered_tessellation = tessellation_a

In [7]:
buffered_tessellation

,uID,geometry
0,0,"POLYGON ((245279.949 702431.270, 245279.687 70..."
1,1,"POLYGON ((261473.357 702178.602, 261473.145 70..."
2,2,"POLYGON ((251870.737 700273.624, 251870.732 70..."
3,3,"POLYGON ((251985.763 701038.463, 251986.526 70..."
4,4,"POLYGON ((261016.516 703116.734, 261016.536 70..."
...,...,...
550835,550835,"POLYGON ((262703.238 653300.470, 262703.257 65..."
550836,550836,"POLYGON ((271182.942 645456.245, 271182.892 64..."
550837,550837,"POLYGON ((261659.702 651822.408, 261659.749 65..."
550838,550838,"POLYGON ((290057.373 642676.912, 290057.361 64..."


In [8]:
tessellation.shape[0]

550840

In [9]:
# def find_neighbours(tessellation, cell):
#     # Get 'not disjoint' countries
#     neighbours = tessellation[~tessellation.geometry.disjoint(cell.geometry)].uID.tolist()
#     # Remove own uID of the cell from the list
#     neighbours_list = [str(uID) for uID in neighbours if cell.uID != uID]
#     return neighbours_list

In [10]:
# out = []

# for index, row in tqdm(tessellation.iterrows(), total=tessellation.shape[0]):
#     out.append(find_neighbours(tessellation, row))

In [11]:
tessellation_scattered = client.scatter(buffered_tessellation)

In [24]:
@dask.delayed
def find_neighbours(tessellation, cell):
    # Get 'not disjoint' countries
    # neighbours = tessellation[~tessellation.geometry.disjoint(cell.geometry)].uID.tolist()
    # # Remove own uID of the cell from the list
    # neighbours_list = [str(uID) for uID in neighbours if cell.uID != uID]
    return tessellation

In [23]:
buffered_tessellation.iloc[0]

uID                                                         0
geometry    POLYGON ((245279.94883140517 702431.2703199963...
Name: 0, dtype: object

In [31]:
@dask.delayed
def find_neighbours(tessellation, cell):
    # Get 'not disjoint' countries
    neighbours = tessellation[~tessellation.geometry.disjoint(cell.geometry)].uID.tolist()
    # # Remove own uID of the cell from the list
    neighbours_list = [str(uID) for uID in neighbours if cell.uID != uID]
    return tessellation

In [35]:
import dask_geopandas as dgp

def find_neighbours(tessellation, cell_id):
    # Ensure tessellation is a Dask-GeoPandas GeoDataFrame
    if not isinstance(tessellation, dgp.GeoDataFrame):
        raise ValueError("tessellation must be a dask_geopandas.GeoDataFrame")

    # Extract the geometry of the specific cell using its unique ID (uID)
    cell_geometry = tessellation[tessellation['uID'] == cell_id].geometry.compute().iloc[0]

    # Filter the tessellation to find neighbours
    # This involves a spatial join where we find all geometries in the tessellation that intersect with the cell_geometry
    neighbours = dgp.sjoin(tessellation, dgp.GeoDataFrame(geometry=[cell_geometry]), how='inner', op='intersects')

    # Remove the original cell from the results
    neighbours = neighbours[neighbours['uID'] != cell_id]

    return neighbours

ModuleNotFoundError: No module named 'dask_geopandas'

In [33]:
tessellation

,uID,geometry
0,0,"POLYGON ((245279.949 702431.270, 245279.687 70..."
1,1,"POLYGON ((261473.357 702178.602, 261473.145 70..."
2,2,"POLYGON ((251870.737 700273.624, 251870.732 70..."
3,3,"POLYGON ((251985.763 701038.463, 251986.526 70..."
4,4,"POLYGON ((261016.516 703116.734, 261016.536 70..."
...,...,...
550835,550835,"POLYGON ((262703.238 653300.470, 262703.257 65..."
550836,550836,"POLYGON ((271182.942 645456.245, 271182.892 64..."
550837,550837,"POLYGON ((261659.702 651822.408, 261659.749 65..."
550838,550838,"POLYGON ((290057.373 642676.912, 290057.361 64..."


In [34]:
test = find_neighbours(buffered_tessellation, buffered_tessellation.iloc[0])
test.compute()

/home/virgilxw/miniconda3/envs/morph_model_v1_tess/lib/python3.11/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 2.42 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2023-11-21 17:27:29,953 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/home/virgilxw/miniconda3/envs/morph_model_v1_tess/lib/python3.11/site-packages/distributed/protocol/core.py", line 160, in loads
    return msgpack.loads(
           ^^^^^^^^^^^^^^
  File "msgpack/_unpacker.pyx", line 194, in msgpack._cmsgpack.unpackb
ValueError: 2586563335 exceeds max_bin_len(2147483647)
2023-11-21 17:27:29,957 - distributed.core - ERROR - Exception while handling op register-client
Traceback (most recent call last):
  File "/home/virgilxw/miniconda3/envs/morph_model_v1_tess/lib/python3.11/site-packages/distributed/core.py", line 968, in _handle_comm
    result = await result
    

CancelledError: find_neighbours-bc1d310f-f14d-47f8-bcc7-e6ff2d8aa6c9

In [14]:
# Constants
BATCH_SIZE = 32

out = []

# Function to process a batch
def process_batch(batch):
    # Compute the batch using Dask
    computed_batch = dask.compute(batch)
    # Update the original DataFrame with the results
    out.append(computed_batch)

# List to hold the current batch of delayed objects
current_batch = []

for index, cell in tessellation.iterrows():
    # Add the delayed task to the current batch
    result = find_neighbours(tessellation_scattered, cell)
    current_batch.append(result)

    # Check if the batch size is reached
    if len(current_batch) >= BATCH_SIZE:
        # Process the current batch
        batch_out = process_batch(current_batch)
        # Reset the batch
        current_batch = []

# Process any remaining tasks in the last batch
if current_batch:
    out.append(process_batch(current_batch))

CancelledError: find_neighbours-e5955831-790f-4e2b-a3de-a11a131b7f02